<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_pretrain_timm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [3]:
# search file paths
from glob import glob


img_paths_train = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png') # *: all
img_paths_train = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png') # *: all
img_paths_val = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/val_50/*.png')
# img_paths_train = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png')) # *: all

In [4]:
# number of images
len(img_paths_train), len(img_paths_val)

(22500, 2500)

In [5]:
import torch
from PIL import Image
import numpy as np


class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, img_size, transform):
        self.img_paths = img_paths
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = Image.open(path).convert('RGB') # read img
        # img_pil = Image.fromarray(img)
        # img = img_pil.resize((IMG_SIZE, IMG_SIZE), Image.LANCZOS)

        # transform img
        img = self.transform(img)

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])
        cls_idx = torch.tensor(cls_idx, dtype=torch.int64)

        return img, cls_idx

In [ ]:
!pip install timm
import timm


# Preprocess Transform
model = timm.create_model('eva02_large_patch14_448.mim_m38m_ft_in1k',
                          pretrained=True,
                          num_classes=50)

data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg)

In [7]:
# classifier head
print(model.get_classifier())
print(model.get_classifier().in_features)

Linear(in_features=1024, out_features=50, bias=True)
1024


In [8]:
# Build Dataset
IMG_SIZE = 32
train_ds = PretrainDataset(img_paths_train, IMG_SIZE, transform)
val_ds = PretrainDataset(img_paths_val, IMG_SIZE, transform)

In [9]:
transform

Compose(
    Resize(size=448, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(448, 448))
    ToTensor()
    Normalize(mean=tensor([0.4815, 0.4578, 0.4082]), std=tensor([0.2686, 0.2613, 0.2758]))
)

In [10]:
# Build DataLoaders
BATCH_SIZE = 300
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE, shuffle=False)

#### Build Model (https://pytorch.org/vision/stable/models.html#classification)

#### Train Model

In [11]:
from tqdm.auto import tqdm # (optional) progress bar


def training(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode. (The Dropout layer has a different behavior between the training mode and the evaluation mode.)
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def testing(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [12]:
from pickle import TRUE
import matplotlib.pyplot as plt
from torch import nn


def train(pretrained=True, freeze=False):
    print(f"pretrained weights: {pretrained}, freeze: {freeze}")
    # Model
    if pretrained:
        model = timm.create_model('eva02_large_patch14_448.mim_m38m_ft_in1k',
                          pretrained=True,
                          num_classes=50)
    else:
        model = timm.create_model('eva02_large_patch14_448.mim_m38m_ft_in1k',
                          pretrained=False,
                          num_classes=50)

    if freeze:
        # Freeze model
        for param in model.parameters():
            param.requires_grad = False

    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    # lower learning rate for finetuning
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=1e-4*0.5)
    # schedule = ReduceLROnplateau(optimizer, 'min')

    EPOCHS = 30
    logs = {
        'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
    }
    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = training(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc = testing(val_loader, model, loss_fn)

        # # Note that step should be called after test_epoch()
        # schedule.step(val_loss)

        print(f'EPOCH: {epoch:04d} \
        train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
        val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

        logs['train_loss'].append(train_loss)
        logs['train_acc'].append(train_acc)
        logs['val_loss'].append(val_loss)
        logs['val_acc'].append(val_acc)


    # plot result
    plt.figure(figsize=(6, 3))
    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.plot(logs['train_loss'])
    plt.plot(logs['val_loss'])
    plt.subplot(1, 2, 2)
    plt.title('Acc.')
    plt.plot(logs['train_acc'])
    plt.plot(logs['val_acc'])
    plt.show()

    # Save model
    torch.save(model.state_dict(), "last.pth")

In [ ]:
# (Not Use)train(pretrained=False)

# transfer learning
train(pretrained=True)
# (Not Use)train(pretrained=True, freeze=True)

pretrained weights: True, freeze: False


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]